In [2]:
from copy import deepcopy
from dataclasses import dataclass
import multiprocessing as mp
from re import T
import torchvision.models as models
import torch
import copy
import torch.nn.functional as F
#from model import net
import random
import multiprocessing as mp
import os
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torchvision import datasets,transforms
import random
from torch import optim,ceil
import torch.nn.functional as F
from torch import nn
import pickle
import numpy as np
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from pandas import DataFrame

In [3]:

from torch.utils.data import Dataset,DataLoader
import numpy as np

class plantVillage(Dataset):
    def __init__(self, df,
                  img_dir='data/oneFolder'
                 , isTest=False):
        self.img_labels=df
        self.img_dir='../../Data/PlantVillage/PlantVillage-Dataset-master/All/resized512/'
       # self.train, self.test = train_test_split(self.img_labels, test_size=0.05)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.img_labels['image'][idx])

            image = (read_image(img_path).float())
            self.transform=transforms.Compose([transforms.Resize((256,256)),
        transforms.Normalize((-0.7898, -1.4373, -0.6051), (1.3209, 1.5867, 1.1863))])
            image = self.transform(image)
            label = self.img_labels['disease'][idx]
            return image, label

In [4]:
df=pd.read_csv('./plantVillage.csv')
df: DataFrame=df.drop('Unnamed: 0',axis=1)
train,valid_test=train_test_split(df,test_size=0.2)
valid,test=train_test_split(valid_test,test_size=0.5)

train_data=plantVillage(df=train.reset_index())
test_data=plantVillage(df=test.reset_index())
valid_data=plantVillage(df=valid.reset_index())

model=models.mobilenet_v2(pretrained=True)
model.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=21, bias=True))

In [5]:
from tqdm import tqdm
class Client():
    def __init__(self,id,dataset,model,config) -> object:
        self.id=id
        self.dataset=dataset
        self.model=model
        self.criterion=torch.nn.CrossEntropyLoss()
        
        self.config=config
    def __len__(self) -> int:
        return len(self.dataset)
    def __str__(self) -> str:
        print('Client ID: {}', self.id)
        
    def get_config(self):
        print(self.config)
        return self.config
    
    def update_model_params(self,params):
        self.model.load_state_dict(params)

    def metric(self,pred,label):
        metric=MulticlassAccuracy(num_classes=21)
        
        return metric(pred,label).item()

    def wandb_log(self,log={}):
        try:
            wandb.log(log)
        except:
            print('Wandb logging error {}', self.id)        
    
    def train_gpu(self,round):
        optimizer=optim.Adam(self.model.parameters(),lr=self.config['learning_rate'],weight_decay=self.config['weight_decay'])
        dataloader=torch.utils.data.DataLoader(self.dataset,
                                         batch_size=int(self.config['batch_size']),
                                         shuffle=True)

        train_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.model.to(train_device)

        print('Client {} start Training'.format(self.id))

        run=wandb.init(group="federated_round: "+str(round), job_type="Client ID: "+str(self.id), project=PROJNAME)
        
        for epoc in (range(self.config['epoch'])):
            epoch_loss=0.0
            step=0
            epoc_accuracy=[]
            
            for x,y in iter(dataloader):
                x,y=x.to(train_device),y.to(train_device)
                output=self.model(x)

                loss=self.criterion(pred,y)

                x,y=x.cpu(),y.cpu()
                output=output.cpu().detach()
                epoch_loss+=loss.item()
                step+=1
                
                pred=output.argmax(dim=1,keepdim=True).T[0]
                epoc_accuracy.append(self.metric(pred,y))
                

                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                optimizer.step()

            accuracy=np.mean(epoc_accuracy)
            print('Epoch : {} , loss: {}, accuracy: {}'.format(epoc,(epoch_loss/step),accuracy))

            self.wandb_log({
                            'epoc':epoc,
                            'loss':(epoch_loss/step),
                            'accuracy':accuracy
                            })
        self.model=self.model.cpu()
        
        run.finish()
        

In [6]:
from torchmetrics.classification import MulticlassAccuracy

class Server():
    def __init__(self,model=models.mobilenet_v2(pretrained=True),dataset=None,eval_dataset=None,config={}):
        self.client_list=[]
        self.model= model
        self.model.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=21, bias=True))
        
        self.dataset=dataset
        self.eval_dataset=eval_dataset
        self.round_dices=[]

        self.best_accuracy=0.0
        
        self.config=config
    def __len__(self) -> int:
        return len(self.client_list)

    def get_config(self):
        print(self.config)
        return self.config

    def split_dataset_1label(self):
        list_dataset=np.array_split(self.dataset,self.config['server']['num_client'])
        list_dataset=map(lambda df:  plantVillage(df.reset_index(drop=True)),list_dataset)
        return list_dataset
    
    def generate_clients(self):
        dataset=self.split_dataset_1label()
        id=0
        for shard in dataset:
            self.client_list.append(Client(id=id,model=copy.deepcopy(self.model),dataset=shard,config=self.config['client']))
            id+=1

    def collect_clients_params(self):
        client_params=[]
        for index in range(0,len(self.round_dices)):
            client_params.append(self.client_list[index].model.state_dict())
        return client_params

    def update_client_model(self,params):
        for client in self.client_list:
            client.model.load_state_dict(copy.deepcopy(params))
        self.model.load_state_dict(copy.deepcopy(params))
        print('Clients updated ! Ready for next round')    

    def fedAvg(self):
        total_len=0
        for index in self.round_dices:
            total_len+=len(self.client_list[index])

        global_model_weights = copy.deepcopy(self.model.state_dict())
        for key in global_model_weights:
            global_model_weights[key] = torch.zeros_like(
                global_model_weights[key])
        
        #server_params=copy.deepcopy(self.centralModel.parameters())

        coef=torch.tensor([len(self.client_list[index])/total_len for index in self.round_dices])
        for index in range(0,len(self.round_dices)):
            local_model_weights = copy.deepcopy(self.client_list[index].model.state_dict())
            for key in global_model_weights:
                if str(global_model_weights[key].type())!='torch.LongTensor':

                    global_model_weights[key] += (coef[index] * local_model_weights[key])
        return global_model_weights

    def metric(self,pred,label):
        metric=MulticlassAccuracy(num_classes=21)
        
        return metric(pred,label).item()

    def wandb_log(self,log={}):
        try:
            wandb.log(log)
        except:
            print('Wandb logging error server', )
            
    def evaluate(self,dataset,round):
        self.model.eval()
        criterion=torch.nn.CrossEntropyLoss()

        dataloader=torch.utils.data.DataLoader(dataset,batch_size=int(10),shuffle=True)
        run=wandb.init(group="federated_Evaluation",job_type='Evaluation', project=PROJNAME)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        test_loss=0.0
        step=0
        accuracyHist=[]
        self.model=self.model.to(device)

        with torch.no_grad():
            for x,y in dataloader:
                x,y=x.to(device),y.to(device)
                output=self.model(x)
                loss=criterion(pred,y)

                
                x,y=x.cpu(),y.cpu()
                output=output.cpu().detach()
                
                test_loss+=loss.item()
                step+=1
                pred=output.argmax(dim=1,keepdim=True).T[0]
                accuracyHist.append(self.metric(pred,y))

        accuracy=np.mean(accuracyHist)
        loss=test_loss/step
        if(accuracy>self.best_accuracy):
            model_log = wandb.Artifact('best_eval_accuracy', type='model')
            try:
                self.best_accuracy=accuracy
                model_log.add_file(self.save_model())
                run.log_artifact(model_log)
                print('model saved, best accuracy: {}'.format(self.best_accuracy))
            except:
                print('save model failed')
        print('Evaluation successfully, ')
        print('Round : {} ,Accuracy : {} loss : {} '.format(round,accuracy,loss))
        self.wandb_log({
                'round':round,
                'loss':loss,
                'accuracy':accuracy
                })
        self.model=self.model.to('cpu')
        return accuracy
    
    def save_model(self):
        file_name='bestmodel.h5'
        torch.save(self.model.state_dict(), file_name)
        return file_name
    
    def round_train(self):
        from random import sample
        for round in range(0,self.config['server']['num_round']):
            
            self.round_dices=sample(range(0,len(self)),self.config['server']['num_client_round'])
            print('Round {}',format(round))
            print('Including clients: {}'.format(self.round_dices))
            for index in self.round_dices:
                self.client_list[index].train_gpu(round=round)
            print('Finish training. Collecting parameters...')
            self.collect_clients_params()
            params=self.fedAvg()
            self.update_client_model(params=params)
            acc=self.evaluate(self.eval_dataset,round=round)

                
            
            
                

In [7]:

CONFIG={
    'client':{
    'learning_rate':0.00006824,
    'batch_size':16,
    'weight_decay':0.000001,
    'epoch':1,
    },
    'server':{
        'num_client':100,
        'num_client_round':3,
        'num_round':50,
    }
}



In [8]:
os.environ["WANDB_API_KEY"] = '761f77141558010f4d706f726fcf122aac0aae23'
os.environ["WANDB_MODE"]="offline"

PROJNAME='fed_train19'
import wandb
df=pd.read_csv('./plantVillage.csv')
df: DataFrame=df.drop('Unnamed: 0',axis=1)
train,valid_test=train_test_split(df,test_size=0.2)
valid,test=train_test_split(valid_test,test_size=0.5)

train_data=plantVillage(df=train.reset_index())
test_data=plantVillage(df=test.reset_index())
valid_data=plantVillage(df=valid.reset_index())

model=models.mobilenet_v2(pretrained=True)
model.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=21, bias=True))


# note that we define values from `wandb.config` instead 
# of defining hard values

config = {
    "lr": 0.00006824,
    "batch_size":16,
    "epoch":20
}
import warnings 
warnings.filterwarnings('ignore')
train=train.reset_index(drop=True)
ser=Server(dataset=train,eval_dataset=test_data,config=CONFIG)
ser.generate_clients()
ser.round_train()



Round {} 0
Including clients: [62, 55, 57]
Client 62 start Training


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch : 0 , loss: 1.267372236468575, accuracy: 0.04061327645216476


accuracy,▁
epoc,▁
loss,▁
accuracy,0.04061
epoc,0
loss,1.26737


Client 55 start Training


...

accuracy,▁
epoc,▁
loss,▁
accuracy,0.04067
epoc,0
loss,1.27197


Finish training. Collecting parameters...
Clients updated ! Ready for next round


model saved, best accuracy: 0.0028694090866093914
Evaluation successfully, 
Round : 0 ,Accuracy : 0.0028694090866093914 loss : 3.524852437718592 
Round {} 1
Including clients: [35, 36, 62]
Client 35 start Training


accuracy,▁
loss,▁
round,▁
accuracy,0.00287
loss,3.52485
round,0


KeyboardInterrupt: 

In [ ]:
tensor([[[ 0.0132, -0.0043,  0.0148],
         [ 0.0328, -0.0254,  0.0069],
         [ 0.0105, -0.0373, -0.0147]],

        [[ 0.0080, -0.0059,  0.0151],
         [ 0.0200, -0.0329, -0.0021],
         [ 0.0114, -0.0330, -0.0079]],

        [[-0.0252, -0.0202, -0.0100],
         [-0.0112, -0.0293, -0.0152],
         [-0.0265, -0.0334, -0.0242]]])

NameError: name 'tensor' is not defined

In [ ]:
ser.client_list[2].model.state_dict()['features.0.0.weight'][0]

tensor([[[ 0.0142, -0.0026,  0.0154],
         [ 0.0329, -0.0226,  0.0078],
         [ 0.0119, -0.0340, -0.0124]],

        [[ 0.0086, -0.0047,  0.0139],
         [ 0.0195, -0.0303, -0.0018],
         [ 0.0117, -0.0305, -0.0070]],

        [[-0.0226, -0.0172, -0.0079],
         [-0.0095, -0.0260, -0.0126],
         [-0.0241, -0.0300, -0.0211]]])

In [ ]:
ser

NameError: name 'ser' is not defined

In [ ]:
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()


free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 16% | 50% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 16% |  8% |
